# Hearthstone Project
***
***

# Goals
- Explore the data to gather insights about the characteristics of the game's different class types

# Setup
***

In [1]:
# establishing environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Acquire
Acquiring data from local csv files
***

In [2]:
# reading in card data and saving as DF
cards = pd.read_csv('hearthstone_standard_cards.csv')

# reading in card classes and saving as DF
classes = pd.read_csv('classes.csv')

# reading in minion types data and saving as DF
mtypes = pd.read_csv('minionTypes.csv')

# reading in rarities data and saving as DF
rarities = pd.read_csv('rarities.csv')

# reading in set groups data and saving as DF
setgroups = pd.read_csv('setGroups.csv')

# reading in card sets data and saving as DF
sets = pd.read_csv('sets.csv')

# reading in types data and saving as DF
ctypes = pd.read_csv('types.csv')

# reading in keywords data and saving as DF
keywords = pd.read_csv('keywords.csv')

# Prepare
Preparing data for exploration
***

## Preparing DataFrames

### Lowercasing all DF columns

In [3]:
# lowercasing cards DF columns
cards.columns = cards.columns.str.lower()

# lowercasing name and text column values
cards.text = cards.text.str.lower()
cards.name = cards.name.str.lower()

# creating list of all DFs besides cards
df_list = [classes, mtypes, rarities, setgroups, sets, ctypes, keywords]

# iterating through DFs
# lowercasing all column names, dropping original name column, renaming slug to name column
for dtafrm in df_list:
        dtafrm.columns = dtafrm.columns.str.lower()
        dtafrm.drop(columns = 'name', inplace = True)
        dtafrm.rename(columns = {"slug": "name"}, inplace = True)       

## Merging DataFrames

### Merging 'classes' DF

In [4]:
# removing brackets and commas from multiclassids column
cards.multiclassids = cards.multiclassids.str.replace('\]|,|\[' , '')

# creating column to hold primary class id 
# if card is of one class, this will reflect its sole class
# if card is dual, this will reflect the 1st of the two classes in the multiClassIds column
# necessary since dual class cards erroneously hold the 'neutral' class value in their primary class id 
cards['primeclassid'] = np.where((cards.multiclassids.str.contains(' ')), cards["multiclassids"].str.split(" ", expand = True)[0], cards.classid)

# converting key columns to make all value data types match
cards.primeclassid = cards.primeclassid.astype(str)
classes.id = classes.id.astype(str)

# merging 'classes' df with card df
df = pd.merge(cards, classes[['id', 'name']], 
              left_on = 'primeclassid', right_on = 'id', how="left", 
              suffixes = (None, '_prime_hero_class'))

# dropping columns I no longer need
df.drop(columns = ['primeclassid', 'classid'], inplace = True)

### Merging 'mtypes' DF

In [5]:
# changing null values of minionTypeId for neutral minions to -1
df['miniontypeid'] = np.where((df.miniontypeid.isnull() == True) & (df.cardtypeid == 4), -1, df.miniontypeid)

# adding missing keyword data to 'keywords' df
# -1 is for minions with no tribe
mtypes.loc[len(mtypes.index)] = ['no_tribe', -1]

# merging 'mtypes' df
df = pd.merge(df, mtypes[['id', 'name']], 
              left_on = 'miniontypeid', right_on = 'id', how="left", 
              suffixes = (None, '_minion_type'))

# dropping column I no longer need
df.drop(columns = ['miniontypeid'], inplace = True)

### Merging 'rarities' DF

In [6]:
# merging 'rarities' df
df = pd.merge(df, rarities[['id', 'name']], 
              left_on = 'rarityid', right_on = 'id', how="left", 
              suffixes = (None, '_rarity'))

# dropping column I no longer need
df.drop(columns = ['rarityid'], inplace = True)

### Merging 'setGroups' DF

In [7]:
# replacing dashes with underscores in names
sets.name = sets.name.str.replace('-', '_')

# merging 'setgroups' df
df = pd.merge(df, sets[['id', 'name']], 
              left_on = 'cardsetid', right_on = 'id', how="left", 
              suffixes = (None, '_set'))

# dropping column I no longer need
df.drop(columns = ['cardsetid'], inplace = True)

### Merging 'ctypes' DF

In [8]:
# merging 'ctypes' df
df = pd.merge(df, ctypes[['id', 'name']], 
              left_on = 'cardtypeid', right_on = 'id', how="left", 
              suffixes = (None, '_card_type'))

# dropping column I no longer need
df.drop(columns = ['cardtypeid'], inplace = True)

### Merging 'keywords' DF

In [9]:
# replacing dashes with underscores in names
keywords.name = keywords.name.str.replace('-', '_')

# adding missing keyword data to 'keywords' df
keywords.loc[len(keywords.index)] = ['64', 'start_of_game', 
                                     'does something at the start of the game.', 
                                     'does something at the start of the game.']

# removing brackets and commas from keyword id column
df.keywordids = df.keywordids.str.replace('\]|,|\[' , '')

# splitting keyword ids into separate columns for each card
kwdf = df["keywordids"].str.split(" ", expand = True) 

# renaming columns
kwdf.columns = ['keywordid1', 'keywordid2', 'keywordid3', 'keywordid4', 'keywordid5']

# concatenating split keyword id columns with main df
df = pd.concat([df, kwdf], axis=1)

# converting keywords id column to str type to enable merge
keywords.id = keywords.id.astype(str)

# creating loop to add a column for the text name of each keyword ability of each card
# via merging with keywords DF
for x in kwdf.columns:
    df = pd.merge(df, keywords[['id', 'name']], 
              left_on = x, right_on = 'id', how = "left",
              suffixes = (None, x + '_name'))

## Checking for duplicate rows

In [10]:
# checking number of rows in current DF
df.shape

(1289, 44)

In [11]:
# Checking number of rows if duplicates were dropped
df.drop_duplicates().shape

(1289, 44)

- No duplicates found

### Checking for proper data types, categorical columns (based on domain knowledge), and null counts

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1289 entries, 0 to 1288
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1289 non-null   int64  
 1   collectible            1289 non-null   int64  
 2   slug                   1289 non-null   object 
 3   multiclassids          1289 non-null   object 
 4   artistname             1288 non-null   object 
 5   manacost               1289 non-null   int64  
 6   name                   1289 non-null   object 
 7   text                   1271 non-null   object 
 8   image                  1289 non-null   object 
 9   imagegold              805 non-null    object 
 10  flavortext             1289 non-null   object 
 11  cropimage              1289 non-null   object 
 12  duels                  708 non-null    object 
 13  health                 825 non-null    float64
 14  attack                 860 non-null    float64
 15  keyw

- The following columns will be dropped as they won't be needed for the expected operations of this project
    - id, slug
        - unique identifiers for cards, not needed since the 'name' column provides this while also being easier to reference
    - artistname, image, imagegold, cropimage
        - I won't be exploring images or artist names in this iteration of the project
    - collectible
        - Only 1 value, no nulls, doesn't distinguish any cards
    - all columns reflecting key words with the exception of the boolean columns and the 'slug_keyword#_name' columns
        - The exempted columns are sufficient for the project's expected operations
     
     
- Based on my domain knowledge of the game, I'm inferring that several of the columns are categorical
    - I need to create boolean columns for categorical columns (rarity, card set, etc.)


- Many null values that need to be addressed
    - text
    - duels
    - minion type id
    - health
    - attack
    - child ids
    - durability
    - armor

### Dropping columns that aren't needed for the planned operations of this project

In [13]:
# creating list of columns to drop
columns_to_drop = ['id', 'slug', 'artistname', 'image', 'imagegold', 'flavortext', 'cropimage', 'collectible']

# dropping columns
df.drop(columns = columns_to_drop, inplace = True)

# Addressing Null Values

### Addressing nulls in 'text' column

In [14]:
# checking values in text box
df.text.value_counts(dropna = False)

NaN                                                                                               18
<b>taunt</b>                                                                                      15
<b>charge</b>                                                                                      7
<b>stealth</b>                                                                                     6
<b>spell damage +1</b>                                                                             6
                                                                                                  ..
<b>battlecry:</b> draw galakrond. if you're already galakrond, unleash a devastation.              1
at the start of your turn, swap this minion with a random one in your hand.                        1
<b>battlecry:</b> restore both heroes to full health.                                              1
<b>battlecry:</b> for each spell you've cast this turn, add a random mage spell to your han

In [15]:
# filling null text values with 'no effect'
df["text"].fillna("no effect", inplace = True) 

### Addressing nulls in 'duels' column

In [16]:
# checking duels values
df.duels.value_counts(dropna = False)

{'relevant': True, 'constructed': True}    708
NaN                                        581
Name: duels, dtype: int64

In [17]:
# updating duels column so that cards that were allowed in duels have value of 1 and 0 otherwise
df['duels'] = np.where((df.duels == "{'relevant': True, 'constructed': True}"), 1, 0)

df.rename(columns={'duels':'in_duels'}, inplace=True)

### Addressing nulls in 'id_minion_type', and 'slug_minion_type' columns

In [18]:
# checking minontypeId values
df.id_minion_type.value_counts(dropna = False)

-1.0     497
 NaN     471
 20.0     75
 15.0     63
 18.0     51
 24.0     50
 17.0     32
 14.0     25
 23.0     18
 21.0      6
 26.0      1
Name: id_minion_type, dtype: int64

In [19]:
# converting nulls, aka non-minion cards to 'not a minion' type
df['id_minion_tribe'] = np.where((df.id_minion_type.isnull() == True), 'not a minion', df.id_minion_type)
df['name_minion_tribe'] = np.where((df.name_minion_type.isnull() == True), 'not a minion', df.name_minion_type)

# dropping minionTypeId since id_minion_type suffices
df.drop(columns = ['id_minion_type', 'name_minion_type'], inplace = True)

### Addressing nulls in 'childIds' column

In [20]:
# checking childIds values
df.childids.value_counts(dropna = False)

NaN                                                                       994
[52897, 52900, 53160, 53161, 53162, 53163, 55378, 60588, 64652, 64653]     15
[59723]                                                                     8
[53921]                                                                     3
[56927]                                                                     3
                                                                         ... 
[53923]                                                                     1
[53577, 53578, 53579]                                                       1
[460]                                                                       1
[61449, 61450]                                                              1
[55778]                                                                     1
Name: childids, Length: 261, dtype: int64

In [21]:
# filling nulls with "no_childid"
df.childids.fillna("no_childid", inplace = True) 

### Addressing nulls in 'health', 'attack', 'durability', and 'armor' columns
All of these variables respective columns have null values since none of these variables apply to every card (examples: only minions have health while only weapons have durability). For the time being I'll fill these nulls 
with a value that represents infinity. The benefit of this method is that it allows me to fill the nulls while preserving the int64 data type of the column. Furthermore, no matter what value blizzard assigns to these variables in future cards, this value probably wouldn't be used. If this causes issues later I'll employ a different means of handling them. 

In [22]:
# creating list of column names
hada = ['health', 'attack', 'durability', 'armor']

# iterating through columns filling nulls within each
for att in hada:
    df[att].fillna(float('inf'), inplace = True)

# Creating boolean columns for categorical variables

### Creating boolean columns for 'keywords'

In [23]:
# loop iterates through each keyword and creates a boolean column for it
for kw in keywords.name:
    df['has_' + kw] = np.where(
    (df.namekeywordid1_name == kw) |
    (df.namekeywordid2_name == kw) |
    (df.namekeywordid3_name == kw) |
    (df.namekeywordid4_name == kw) |
    (df.namekeywordid5_name == kw), 1, 0)
    
# creating empty list
key_word_col_drop = []

# iterating through columns in df and creating list of columns to drop
for col in df.columns:
    if 'keywordid' in col:
        key_word_col_drop.append(col)
        
# dropping columns
df.drop(columns = key_word_col_drop, inplace = True)

### Creating boolean columns for 'hero classes'

In [24]:
# removing brackets and commas from multiclassids column
df.multiclassids = df.multiclassids.str.replace('\]|,|\[' , '')

# creating column that holds secondary class separate from primary class
df['id_second_hero_class'] = df["multiclassids"].str.split(" ", expand = True)[1]

# converting column to str type to enable merge with newly created column 'id_second_hero_class'
classes.id = classes.id.astype(str)

# creating df containing columns for merge in order to rename before merge without altering original classes DF
classes2 = classes[['id', 'name']]

# renaming columns
classes2.columns = ['id_second_hero_class', 'name_second_hero_class']

# merging 'classes' on secondary hero class id to get secondary class names
df = pd.merge(df, classes2[['id_second_hero_class', 'name_second_hero_class']], 
              on = 'id_second_hero_class', how = "left")

# creating boolean columns for each hero class
for c in classes.name:
    df['is_' + c] = np.where(
    (df.name_prime_hero_class == c) | (df.name_second_hero_class == c), 1, 0)

# filling nulls in new columns
df['name_second_hero_class'].fillna('monoclass', inplace = True)
df['id_second_hero_class'].fillna('monoclass', inplace = True)

### Creating boolean column for multiclass cards

In [25]:
# creating column where 1 = multiclass, 0 = monoclass)
# contains ' ' will suffice since only cards with a space in this value are multiclass
df['is_multiclass'] = np.where((df.multiclassids.str.contains(' ')), 1, 0)

# dropping column I no longer need
df.drop(columns = 'multiclassids', inplace = True)

### Creating boolean column for cards with child ids

In [26]:
# creating column where 1 = card has childids, 0 = card has no childids)
# contains ',' will suffice since only cards with a comma in this value have childids
df['has_child_ids'] = np.where((df.childids.str.contains(',')), 1, 0)

# dropping column I no longer need
df.drop(columns = 'childids', inplace = True)

### Creating boolean columns for rarity levels

In [27]:
# iterating through levels of rarity (common, rare, epic, etc)
# creating boolean column for each
for level in rarities.name:
    df['is_' + level] = np.where((df.name_rarity == level), 1, 0)

### Creating boolean columns for card sets

In [28]:
# iterating through set names and creating a boolean column for each
for setname in sets.name:
    df['is_' + setname] = np.where((df.name_set == setname), 1, 0)

### Creating boolean columns for card type

In [29]:
# iterating through card types and creating a boolean column for each
for ctype in ctypes.name:
    df['is_' + ctype] = np.where((df.name_card_type == ctype), 1, 0)

# dropping column I no longer need
df.drop(columns = 'id_card_type', inplace = True)

### Creating boolean columns for minion tribe

In [30]:
# iterating through minion tribes and creating a boolean column for each
for mtype in mtypes.name:
    df['is_' + mtype] = np.where((df.name_minion_tribe == mtype), 1, 0)

# dropping column I no longer need
df.drop(columns = 'id_minion_tribe', inplace = True)

### Dropping boolean columns with all 0 values
I know that some of the sets and keywords that were turned into booleans are not currently in standard (the format the collection of the cards in the data are part of). These columns will be completely filled with 0s so I'm going to drop them.

In [31]:
# creating list of index values for columns that only have 0 values
all_0_cols = np.where(df.isin([0]).all() == True)

# dropping columns based on index value
df.drop(df.columns[all_0_cols], axis = 1, inplace = True)

### Adding rows for dual class cards with prime and secondary class swapped
Adding these rows will make it easier to perform certain operations such as grouping and plotting.

In [33]:
# making identical dfs of all dual class cards
dcc = df[df.name_second_hero_class != 'monoclass']
dcc2 = df[df.name_second_hero_class != 'monoclass']

# swapping primary and secondary hero class values
dcc2.name_prime_hero_class, dcc2.name_second_hero_class, dcc2.id_prime_hero_class, dcc2.id_second_hero_class = dcc.name_second_hero_class, dcc.name_prime_hero_class, dcc.id_second_hero_class, dcc.id_prime_hero_class

# adding new rows to main df
df = pd.concat([df, dcc2])

# resetting index
df.reset_index(drop = True, inplace = True)

### Adjusting column order

In [32]:
# adjusting order of columns
df = df[['manacost', 'name', 'text', 'in_duels', 'has_child_ids', 'health', 'attack',
       'durability', 'armor', 'id_prime_hero_class', 'name_prime_hero_class', 
       'id_second_hero_class', 'name_second_hero_class',
       'id_rarity', 'name_rarity', 'id_set', 'name_set', 'name_card_type',
       'name_minion_tribe', 'has_taunt', 'has_spellpower', 'has_divine_shield',
       'has_charge', 'has_secret', 'has_stealth', 'has_battlecry',
       'has_freeze', 'has_windfury', 'has_deathrattle', 'has_combo',
       'has_overload', 'has_silence', 'has_counter', 'has_immune',
       'has_discover', 'has_quest', 'has_poisonous', 'has_lifesteal',
       'has_rush', 'has_evilzug', 'has_twinspell', 'has_mega_windfury',
       'has_reborn', 'has_empower', 'has_outcast', 'has_spellburst',
       'has_sidequest', 'has_corrupt', 'has_start_of_game',
       'is_demonhunter', 'is_druid', 'is_hunter', 'is_mage', 'is_paladin', 'is_priest',
       'is_rogue', 'is_shaman', 'is_warlock', 'is_warrior', 'is_neutral',
       'is_multiclass', 'is_common', 'is_free', 'is_rare',
       'is_epic', 'is_legendary', 'is_madness_at_the_darkmoon_faire',
       'is_scholomance_academy', 'is_demonhunter_initiate',
       'is_ashes_of_outland', 'is_galakronds_awakening',
       'is_descent_of_dragons', 'is_saviors_of_uldum', 'is_rise_of_shadows',
       'is_classic', 'is_basic', 'is_hero', 'is_minion', 'is_spell',
       'is_weapon', 'is_murloc', 'is_demon', 'is_mech', 'is_elemental',
       'is_beast', 'is_totem', 'is_pirate', 'is_dragon', 'is_all',
       'is_no_tribe']]

# Preparation Phase Summary
- Lowercased all column names and values

- Merged all dataFrames into single dataframe

- Added missing keyword value (start of game)

- Changed all dashes in column names to underscores

- Checked for duplicate rows, none found

- Filled nulls with different values based on column and intended operations

- Created boolean columns for categorical values

- Dropped columns that were unuseful for the operations of this project

- Reordered columns

# Explore
Exploring the data to identify the characteristics of each class, including neutral

In [36]:
# distribution of mana cost

mana = df.groupby(['name_prime_hero_class', 'name_second_hero_class']).mean()['manacost']

mana

name_prime_hero_class  name_second_hero_class
demonhunter            hunter                    2.400000
                       monoclass                 3.849315
                       warlock                   2.800000
druid                  hunter                    3.600000
                       monoclass                 3.965116
                       shaman                    2.800000
hunter                 demonhunter               2.400000
                       druid                     3.600000
                       monoclass                 3.430233
mage                   monoclass                 3.720930
                       rogue                     3.200000
                       shaman                    2.200000
neutral                monoclass                 4.020672
paladin                monoclass                 3.500000
                       priest                    3.600000
                       warrior                   3.600000
priest                 mon